In [192]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import os
import json
from config import postgres_pw, postgres_user


In [193]:

csv_file = "C:/Users/gregp/Git_Trilogy/ETL_Project/Resources/aviationdata_1.csv"
aviation_data_df = pd.read_csv(csv_file)
aviation_data_df.head()


,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.Flight,Air.Carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.Phase.of.Flight,Report.Status,Publication.Date
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20191231X83852,Accident,CEN20FA049,12/31/2019,"OLATHE, KS",United States,38.846111,-94.736111,OJC,Johnson County Executive,...,Personal,NaN,2.0,NaN,NaN,NaN,VMC,TAKEOFF,Preliminary,8/1/2020
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [194]:
new_aviation_data_df = aviation_data_df[["Event.Date", "Location", "Country", "Airport.Code", "Total.Fatal.Injuries"]].copy()

new_aviation_data_df

,Event.Date,Location,Country,Airport.Code,Total.Fatal.Injuries
0,NaN,NaN,NaN,NaN,NaN
1,12/31/2019,"OLATHE, KS",United States,OJC,2.0
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
84256,NaN,NaN,NaN,NaN,NaN
84257,NaN,NaN,NaN,NaN,NaN
84258,NaN,NaN,NaN,NaN,NaN
84259,NaN,NaN,NaN,NaN,NaN


In [195]:
new_aviation_data_df = new_aviation_data_df.dropna()
new_aviation_data_df

,Event.Date,Location,Country,Airport.Code,Total.Fatal.Injuries
1,12/31/2019,"OLATHE, KS",United States,OJC,2.0
5,12/28/2019,"LAFAYETTE, LA",United States,LFT,5.0
6,12/25/2019,"Headland, AL",United States,0J6,1.0
10,12/21/2019,"Evansville, IN",United States,EVV,1.0
35,12/1/2019,"San Antonio, TX",United States,SAT,3.0
...,...,...,...,...,...
84245,1/2/1982,"YPSILANTI, MI",United States,YIP,0.0
84246,1/2/1982,"CHARLOTTE, MI",United States,49G,0.0
84247,1/2/1982,"CHAMBLEE, GA",United States,PDK,0.0
84250,1/1/1982,"EAST HANOVER, NJ",United States,N58,0.0


In [196]:
csv_file_2 = "C:/Users/gregp/Git_Trilogy/ETL_Project/Resources/infoaviation.csv"
infoaviation_data_df_2 = pd.read_csv(csv_file_2)
infoaviation_data_df_2.head()

,Date,Time,Location,Operator,Flight #,Route,AC Type,Registration,cn/ln,Aboard,Aboard Passangers,Aboard Crew,Fatalities,Fatalities Passangers,Fatalities Crew,Ground,Summary
0,9/17/1908,17:18,"Fort Myer, Virginia",Military - U.S. Army,NaN,Demonstration,Wright Flyer III,NaN,1,2.0,1.0,1.0,1.0,1.0,0.0,0.0,"During a demonstration flight, a U.S. Army fly..."
1,9/7/1909,NaN,"Juvisy-sur-Orge, France",NaN,NaN,Air show,Wright Byplane,SC1,NaN,1.0,0.0,1.0,1.0,0.0,0.0,0.0,Eugene Lefebvre was the first pilot to ever be...
2,7/12/1912,6:30,"Atlantic City, New Jersey",Military - U.S. Navy,NaN,Test flight,Dirigible,NaN,NaN,5.0,0.0,5.0,5.0,0.0,5.0,0.0,First U.S. dirigible Akron exploded just offsh...
3,8/6/1913,NaN,"Victoria, British Columbia, Canada",Private,NaN,NaN,Curtiss seaplane,NaN,NaN,1.0,0.0,1.0,1.0,0.0,1.0,0.0,The first fatal airplane accident in Canada oc...
4,9/9/1913,18:30,Over the North Sea,Military - German Navy,NaN,NaN,Zeppelin L-1 (airship),NaN,NaN,20.0,NaN,NaN,14.0,NaN,NaN,0.0,The airship flew into a thunderstorm and encou...


In [197]:
infoaviation_data_df_2 = infoaviation_data_df_2[["Date", "Location", "Fatalities", "Summary"]].copy()
infoaviation_data_df_2

,Date,Location,Fatalities,Summary
0,9/17/1908,"Fort Myer, Virginia",1.0,"During a demonstration flight, a U.S. Army fly..."
1,9/7/1909,"Juvisy-sur-Orge, France",1.0,Eugene Lefebvre was the first pilot to ever be...
2,7/12/1912,"Atlantic City, New Jersey",5.0,First U.S. dirigible Akron exploded just offsh...
3,8/6/1913,"Victoria, British Columbia, Canada",1.0,The first fatal airplane accident in Canada oc...
4,9/9/1913,Over the North Sea,14.0,The airship flew into a thunderstorm and encou...
...,...,...,...,...
4962,4/16/2019,"Puerto Montt, Chile",6.0,"While the aircraft was in the initial climb, p..."
4963,5/5/2019,"Near Monclava, Mexico",13.0,The aircraft crashed while en route on a retur...
4964,5/5/2019,"Moscow, Russia",41.0,Forty-five minutes after taking off from Mosco...
4965,6/3/2019,"Near Lipo, India",13.0,Crashed about 34km WNW of Mechuka.


In [198]:
infoaviation_data_df_2 = infoaviation_data_df_2.dropna()
infoaviation_data_df_2

,Date,Location,Fatalities,Summary
0,9/17/1908,"Fort Myer, Virginia",1.0,"During a demonstration flight, a U.S. Army fly..."
1,9/7/1909,"Juvisy-sur-Orge, France",1.0,Eugene Lefebvre was the first pilot to ever be...
2,7/12/1912,"Atlantic City, New Jersey",5.0,First U.S. dirigible Akron exploded just offsh...
3,8/6/1913,"Victoria, British Columbia, Canada",1.0,The first fatal airplane accident in Canada oc...
4,9/9/1913,Over the North Sea,14.0,The airship flew into a thunderstorm and encou...
...,...,...,...,...
4962,4/16/2019,"Puerto Montt, Chile",6.0,"While the aircraft was in the initial climb, p..."
4963,5/5/2019,"Near Monclava, Mexico",13.0,The aircraft crashed while en route on a retur...
4964,5/5/2019,"Moscow, Russia",41.0,Forty-five minutes after taking off from Mosco...
4965,6/3/2019,"Near Lipo, India",13.0,Crashed about 34km WNW of Mechuka.


In [199]:
%%writefile config.py
postgres_user = "postgres"
postgres_pw = "postgres"

Overwriting config.py


In [200]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [201]:
rds_connection_string = (f"{postgres_user}:{postgres_pw}"
                         + "@localhost:5432/customer_db")
engine = create_engine(f'postgresql://{rds_connection_string}')